# Data Set

In [ ]:
from random import random
import numpy as np

In [ ]:
DS = {
    'ntriangles': 0,
    'triangles': {}
}

def new_triangle(node_ids, parents=[], children=[], neighbors=[]):
    DS['ntriangles'] += 1
    triangle_id = DS['ntriangles']
    
    return {
        'id': triangle_id,
        'nodes': node_ids,
        'parents': parents,
        'children': children,
        'neighbors': neighbors,
    }

# Test

In [ ]:
# nodes for testing
vtx = [(0,0), (1,0), (2,0), (0,2), (2, 2), (-1, 3), (0, 4)]
test_nodes = np.zeros(len(vtx), dtype=[('id', np.int), ('lat', np.float64), ('lon', np.float64), ('elev', np.float64)])

for v, node in zip(vtx, test_nodes):
    node['lon'] = v[0]
    node['lat'] = v[1]
    node['elev'] = 6
    
sort_order = np.argsort(test_nodes, order=('lat', 'lon'))
np.take(test_nodes, sort_order, out=test_nodes)

for i, node in enumerate(test_nodes):
    node['id'] = i
    
DS_TEST = {
    'ntriangles': 4,
    'triangles': {
        0: new_triangle(node_ids=[6, -1, -2], parents=[], children=[1, 2, 3], neighbors=[]),
        1: new_triangle(node_ids=[6, -1, 0], parents=[0], children=[], neighbors=[]),
        2: new_triangle(node_ids=[0, -1, -2], parents=[0], children=[], neighbors=[]),
        3: new_triangle(node_ids=[-2, 0, 6], parents=[0], children=[], neighbors=[]),
    }
}


In [ ]:
import os
import requests
from zipfile import ZipFile
import numpy as np
from scipy.interpolate import interp2d


class SRTM3API:

    SRTM3_URL = 'https://dds.cr.usgs.gov/srtm/version2_1/SRTM3'
    SRTM3API_DIR = os.environ['HOME'] + '/.srtm_api'
    SRTM3_RES = 1201
    SRTM3_REGIONS = [
        # TODO: Complete this list
        {'lon': (-15, 180), 'lat': (35, 61), 'code': 'Eurasia'},
        {'lon': (60, 180), 'lat': (-10, 35), 'code': 'Eurasia'},
        {'lon': (-26, 60), 'lat': (-35, 35), 'code': 'Africa'},
    ]

    def __init__(self, area=[]):
        """
        Downloading SRTM3 data from usgs.gov and creating a mesh

        Parameter
        area: (bl_lat, bl_on, tr_lat, tr_lon), bl: bottom left, tr: top right
        """

        # Create srtm_api config directory
        if 'HOME' not in os.environ:
            return

        if not os.path.exists(self.SRTM3API_DIR):
            os.makedirs(self.SRTM3API_DIR)

        # Create region directories inside srtm_api directory
        for region in self.SRTM3_REGIONS:
            dirname = self.SRTM3API_DIR + '/' + region['code']
            if not os.path.exists(dirname):
                os.makedirs(dirname)

        xpixels = (int(area[3] - area[1]) + 1) * self.SRTM3_RES
        ypixels = (int(area[2] - area[0]) + 1) * self.SRTM3_RES

        self.data = {
            'bl': {'lat': int(area[0]), 'lon': int(area[1])},
            'tr': {'lat': int(area[2]) + 1, 'lon': int(area[3]) + 1},
            'resolution': (xpixels, ypixels),
            'mesh': np.zeros((xpixels, ypixels), dtype=np.dtype('>i2'))
        }

        self._download_area()
        self._load_hgt_data()

    def elevation(self, lon, lat, interpolate=True):
        """
        Return the elevation of a given (lon, lat) by interpolating STRM3 data

        Parameter
        lon: longitude
        lat: latitude
        interpolate: If True, uses interpolation to find the elevation
        """

        dlon = float(lon) - self.data['bl']['lon']
        dlat = float(lat) - self.data['bl']['lat']

        x = dlon * self.SRTM3_RES
        y = dlat * self.SRTM3_RES

        i = int(x)
        j = int(y)

        max_i = self.data['resolution'][0]
        max_j = self.data['resolution'][1]

        if interpolate:
            if i - 1.5 < 0 or i + 3.5 > max_i or j - 1.5 < 0 or j + 3.5 > max_j:
                raise RuntimeError('Out of range!', lon, lat)

            xi, yi = np.meshgrid(
                np.arange(i - 1.5, i + 3.5, 1.0),
                np.arange(j - 1.5, j + 3.5, 1.0))
            f = interp2d(
                xi, yi, self.data['mesh'][i-2:i+3, j-2:j+3], kind='cubic')

            return f(x, y)[0]
        else:
            if not 0 <= i < max_i or not 0 <= j < max_j:
                raise RuntimeError('Out of range!', lon, lat)

            return self.data['mesh'][i, j]

    def _download_area(self):
        """
        Downloading SRTM3 hgt files
        """
        for lon in range(self.data['bl']['lon'], self.data['tr']['lon']):
            for lat in range(self.data['bl']['lat'], self.data['tr']['lat']):
                mdata = self._get_metadata(lon, lat)

                if os.path.exists(mdata['file']['path']):
                    continue

                hgt = requests.get(mdata['url'])
                open(mdata['zip']['path'], 'wb').write(hgt.content)

                hgtzip = ZipFile(mdata['zip']['path'], 'r')
                hgtzip.extractall(mdata['directory']['path'])
                hgtzip.close()

    def _get_region_code(self, lon, lat):
        """
        Getting the region name based on (lon, lat)
        """
        for region in self.SRTM3_REGIONS:
            if lon >= region['lon'][0] and lon <= region['lon'][1]:
                if lat >= region['lat'][0] and lat <= region['lat'][1]:
                    return region['code']

        raise RuntimeError('Region not found!', lon, lat)

    def _get_metadata(self, lon, lat):
        regionname = self._get_region_code(lon, lat)

        dirpath = self.SRTM3API_DIR + '/' + regionname

        lon_tag = "%s%03d" % ('E' if int(lon) > 0 else 'W', abs(int(lon)))
        lat_tag = "%s%02d" % ('N' if int(lat) > 0 else 'S', abs(int(lat)))
        filename = lat_tag + lon_tag + '.hgt'

        return {
            'region': regionname,
            'directory': {
                'name': regionname,
                'path': dirpath
            },
            'file': {
                'name': filename,
                'path': dirpath + '/' + filename,
            },
            'zip': {
                'name': filename + '.zip',
                'path': dirpath + '/' + filename + '.zip',
            },
            'url': self.SRTM3_URL + '/' + regionname + '/' + filename + '.zip',
        }

    def _load_hgt_data(self):
        """
        Loading downloaded .hgt files into self.data
        """
        for lon in range(self.data['bl']['lon'], self.data['tr']['lon']):
            for lat in range(self.data['bl']['lat'], self.data['tr']['lat']):
                mdata = self._get_metadata(lon, lat)

                i = (lon - self.data['bl']['lon']) * self.SRTM3_RES
                i_end = i + self.SRTM3_RES

                j = (lat - self.data['bl']['lat']) * self.SRTM3_RES
                j_end = j + self.SRTM3_RES

                # We need to flip the hgt matrix along y-dir since it is
                # arranged from west to east and then **north to south**
                self.data['mesh'][i:i_end, j:j_end] = np.flip(
                    np.fromfile(
                        mdata['file']['path'],
                        np.dtype('>i2'),
                        self.SRTM3_RES*self.SRTM3_RES
                    ).reshape((self.SRTM3_RES, self.SRTM3_RES)), 1  # y-dir
                )

In [ ]:
bl = {'lat': 47.37645735, 'lon': 8.4330368}
tr = {'lat': 47.42727336, 'lon': 8.55671103}

area = [bl['lat'], bl['lon'], tr['lat'], tr['lon']]

srtm = SRTM3API(area)

In [ ]:
N = 6

nodes = np.zeros(N, dtype=[('id', np.int), ('lat', np.float64), ('lon', np.float64), ('elev', np.float64)])

vtx = [(0,0), (1,0), (2,0), (0,2), (2, 2), (0, 4)]

for v, node in zip(vtx, nodes):
    node['lat'] = v[0]
    node['lon'] = v[1]
    node['elev'] = 0
    
sort_order = np.argsort(nodes, order=('lat', 'lon'))
np.take(nodes, sort_order, out=nodes)

for i, node in enumerate(nodes):
    node['id'] = i
    
print(nodes)

## Data Structure

### Triangles:
- Triangle id
- A list of thress node ids,
- Parent triangle(s)
- Child triangle(s)
- Neighbor triangle(s)

## Points in General Position

- if the points are not in General posriton, generates new node which is in general position

In [ ]:
# def not_aligned(arr, lat, lon):
#     if len(arr) == 0:
#         return True

#     for i, n1 in enumerate(arr):
#         for j, n2 in enumerate(arr):
#             if i == j:
#                 continue
#             a1 = n1['lat'] / n1['lon']
#             a2 = n2['lat'] / n2['lon']
#             a = lat / lon

#             if abs(a1 - a2) < 1e-5 and abs(a1 - a) < 1e-5:
#                 return False
#     return True

# for i, node in enumerate(nodes):
#     while(True):
#         lat = bl['lat'] + random() * (tr['lat'] - bl['lat'])
#         lon = bl['lon'] + random() * (tr['lon'] - bl['lon'])
#         if not_aligned(nodes[:i], lat, lon):
#             elev = srtm.elevation(lon, lat)
#             node['lat'] = lat
#             node['lon'] = lon
#             node['elev'] = elev

#             break
            
# def general_position(arr, lat, lon):
#     if len(arr) == 0:
#         return True
    
#     for i, ii in enumerate(arr):
#         for j, jj in enumerate(arr):
#             if i == j:
#                 continue
#             for k, kk in enumerate(arr):
#                 if j == k:
#                     continue
#                 for l, ll in enumerate(arr):
#                     if k == l:
#                         continue
                        
#                     if in_circle([i, j, k], l)[1] == 0:
#                         return False
#     return True

# for i, node in enumerate(nodes):
#     while(True):
#         lat = bl['lat'] + random() * (tr['lat'] - bl['lat'])
#         lon = bl['lon'] + random() * (tr['lon'] - bl['lon'])
#         if not general_position(nodes[:i], lat, lon):
#             elev = srtm.elevation(lon, lat)
#             node['lat'] = lat
#             node['lon'] = lon
#             node['elev'] = elev

#             break

## Lexicographic order of nodes

- Lexicographic ordering of nodes (lat, long)
- Filling node ids into nodes array

## Rooting Triangle
- The initial triangle including node ids  -1, -2

In [ ]:
DS['triangles'][0] = new_triangle(node_ids=[nodes[-1]['id'], -1, -2], parents=None, children=[1,2,3], neighbors=None)
p1 = nodes[0]
DS['triangles'][1] = new_triangle(node_ids=[p1['id'], -1, -2], parents=[0], children=[], neighbors=[2,3])
DS['triangles'][2] = new_triangle(node_ids=[p1['id'], nodes[-1]['id'], -2], parents=[0], children=[], neighbors=[1,3])
DS['triangles'][3] = new_triangle(node_ids=[p1['id'], -1, nodes[-1]['id']], parents=[0], children=[], neighbors=[1,2])
DS['triangles']

## Locate
Locating the triangle which contains a given node

In [ ]:
def det(a, b, c):
    """
    triangle := list of there node_ids
    """    
    array = np.array([
        [a['lon'], a['lat'], 1],
        [b['lon'], b['lat'], 1],
        [c['lon'], c['lat'], 1]
    ])
    
    return np.linalg.det(array)

def find_left_and_right(node1, node2):
    if node1['lon'] == node2['lon']:
        if node1['lat'] < node2['lat']:
            left, right = node1, node2
        else:
            left, right = node2, node1
    elif node1['lon'] < node2['lon']:
        left, right = node1, node2
    else:
        left, right = node2, node1
        
    return left, right

def is_in_triangle(tri, point, nodes):
    node_ids = tri['nodes']
    
    if -1 in node_ids and -2 not in node_ids:
        id1, id2 = list(set(node_ids) - {-1})

        left, right = find_left_and_right(nodes[id1], nodes[id2])

        if point['id'] > left['id'] and point['id'] < right['id'] and det(left, right, point) < 0:
            return True
        else:
            return False
        
    elif -2 in node_ids and -1 not in node_ids:
        id1, id2 = list(set(node_ids) - {-2})
        
        left, right = find_left_and_right(nodes[id1], nodes[id2])

        if point['id'] > left['id'] and point['id'] < right['id'] and det(left, right, point) > 0:
            return True
        else:
            return False
        
    elif -1 in node_ids and -2 in node_ids:
        id1 = list(set(node_ids) - {-1, -2})
        
        if point['id'] < id1:
            return True
        else:
            return False
    else:
        a = nodes[tri['nodes'][0]]
        b = nodes[tri['nodes'][1]]
        c = nodes[tri['nodes'][2]]

        t0 = det(a, b, c)
        t1 = det(a, b, point)
        t2 = det(a, point, c)
        t3 = det(point, b, c)

        if abs(abs(t0) - (abs(t1) + abs(t2) + abs(t3))) < 1e-8:
            return True
        else:
            return False

# Tests
# including only -1
assert is_in_triangle(new_triangle(node_ids=[0, -1, 6], parents=[0], children=[]), test_nodes[1], test_nodes) == True
assert is_in_triangle(new_triangle(node_ids=[0, -1, 6], parents=[0], children=[]), test_nodes[4], test_nodes) == True
assert is_in_triangle(new_triangle(node_ids=[0, 3, -1], parents=[0], children=[]), test_nodes[1], test_nodes) == True
assert is_in_triangle(new_triangle(node_ids=[0, 3, -1], parents=[0], children=[]), test_nodes[4], test_nodes) == False

# including only -2
assert is_in_triangle(new_triangle(node_ids=[6, -2, 0], parents=[0], children=[]), test_nodes[5], test_nodes) == True
assert is_in_triangle(new_triangle(node_ids=[3, -2, 0], parents=[0], children=[]), test_nodes[5], test_nodes) == False

# including both -1 and -2
assert is_in_triangle(new_triangle(node_ids=[-1, -2, 3], parents=[0], children=[]), test_nodes[0], test_nodes) == True
assert is_in_triangle(new_triangle(node_ids=[-1, -2, 3], parents=[0], children=[]), test_nodes[4], test_nodes) == False
assert is_in_triangle(new_triangle(node_ids=[-1, -2, 0], parents=[0], children=[]), test_nodes[4], test_nodes) == False

# not including either -1 or -2
assert is_in_triangle(new_triangle(node_ids=[0, 5, 4], parents=[0], children=[]), test_nodes[3], test_nodes) == True
assert is_in_triangle(new_triangle(node_ids=[0, 5, 4], parents=[0], children=[]), test_nodes[1], test_nodes) == False

In [ ]:
def is_on_an_edge(tri, point, nodes):
    node_ids = tri['nodes']
    
    if -1 in node_ids or -2 in node_ids:
        return False, ()
    
    a = nodes[tri['nodes'][0]]
    b = nodes[tri['nodes'][1]]
    c = nodes[tri['nodes'][2]]
    
    d1 = det(a, b, point)
    d2 = det(a, point, c)
    d3 = det(point, b, c)
    
    if abs(d1) < 1e-8:
        return True, (a['id'], b['id'])
    elif abs(d2) < 1e-8:
        return True, (a['id'], c['id'])
    elif abs(d3) < 1e-8:
        return True, (b['id'], c['id'])
    else:
        return False, ()
    

# tests
assert is_on_an_edge(new_triangle(node_ids=[0, 2, 3], parents=[0], children=[]), test_nodes[1], test_nodes) == (True, (0, 2))
assert is_on_an_edge(new_triangle(node_ids=[0, 2, 3], parents=[0], children=[]), test_nodes[4], test_nodes) == (False, ())

In [ ]:
def find_neighbour(tri, edge, triangles):
    neighbour = None
    
    
def find_neighbour(edge, new_triangle_id):    
    neighbour_id = Node
    
    for parent in DS['triangles'][new_triangle_id]['parents']:
        neighbours = parent['neighbor']
        for neighbour in neighbours:
            if edge[0] and edge[1] in DS['triangles'][neighbours]['nodes']:
                neighbour_id = neighbour
                
    return neighbour_id if neighbour_id else False

In [ ]:
# Locating the triangle which contains a given node
def locate(node_id, nodes, triangles):
    is_here = [0]
    node = nodes[node_id]
    
    children_ids = [c_id for children in [triangles[i]['children'] for i in is_here] for c_id in children]
    
    while children_ids:
        within = []
        for child_id in children_ids:
            child = triangles[child_id]

            if is_in_triangle(child, node, nodes):                
                within.append(child_id)
                
        is_here = within
        children_ids = [c_id for children in [triangles[i]['children'] for i in within] for c_id in children]
        
    return is_here

# tests
assert locate(4, test_nodes, DS_TEST['triangles']) == [1]
assert locate(2, test_nodes, DS_TEST['triangles']) == [1]
assert locate(5, test_nodes, DS_TEST['triangles']) == [3]

## Find the neighbouring triangle sharing an edge
- Find the node id of neighbouring triangle 

In [ ]:
def in_circle(triangle, d):
    
    """
    triangle : list of 3 elements
    d : the new node we added to triangulation
    """
    
    in_circle = []
    in_circle[0] = False
    
    a = triangle[0]
    b = triangle[1]
    c = triangle[2]
    
    if clockwise(a, b, c)[0]:
        array = np.array([
            [a[0], a[1], (a[0]^2) + (a[1]^2), 1],
            [b[0], b[1], (b[0]^2) + (b[1]^2), 1], 
            [c[0], c[1], (c[0]^2) + (c[1]^2), 1],
            [d[0], d[1], (d[0]^2) + (d[1]^2), 1]
        ])
    else:
        array = np.array([
            [a[0], a[1], (a[0]^2) + (a[1]^2), 1],
            [c[0], c[1], (c[0]^2) + (c[1]^2), 1], 
            [b[0], b[1], (b[0]^2) + (b[1]^2), 1],
            [d[0], d[1], (d[0]^2) + (d[1]^2), 1]
        ])
        
    
    det = np.linalg.det(array) 
    if det > 0:
        in_circle[0] = True
        
    return [in_circle, det]

In [ ]:


def in_triangle(triangle, d):
    """
    check if the point lies in a triangle or on one the edge of triangle
    tiangle := list of three nodes ids
    d := the new node
    """
    
    t0 = clockwise(triangle)[1]
    t1 = clockwise([triangle[0], triangle[1], d])[1]
    t2 = clockwise([triangle[0], triangle[2], d])[1]
    t3 = clockwise([triangle[1], triangle[2], d])[1]
    
    if abs(t0) == abs(t1) + abs(t2) + abs(t3):
        
    
        if abs(t0) == abs(t1) + abs(t2) + abs(3):
            return [True, triangle]
        else:
            return [False, []]  



In [ ]:
def share_edge(neighbour_id, triangle_id):
    
    share = False
    
    nb = DS['triangles'][neighbour_id]
    tr = DS['triangles'][triangle_id]
    
    if tr['nodes'][0] and tr['nodes'][1] in nb['nodes']:
        share = True
        return share
        
    elif tr['nodes'][0] and tr['nodes'][2] in nb['nodes']:
        share = True
        return share
    
    elif tr['nodes'][1] and tr['nodes'][2] in nb['nodes']:
        share = True
        return share
    
    else:
        return share    

In [ ]:
def find_common_nodes(T1, T2):
    """
    T1, T2 : lists of node ids of triangle T1 and T2
    """
    common_nodes = []
    
    for node_id in T1:
        if node_id in T2:
            common_nodes.append(node_id)
            
    return common_nodes

## LegalizeEdge function
- flips the illegal edge and make it legal

In [ ]:
def legalize(node_id, edge, triangle_id, DS):
    """
    node_id
    edge := list of two node ids of the edge
    DS := data structure
    """
    always_legal = [-1, -2, nodes[-1]['id']]
    
    if set(edge) == set([-1, -2]) or set(edge) == set([-1, nodes[-1]['id']]) or set(edge) == set([-2, nodes[-1]['id']]):
        return DS
    
    elif node_id > 0 and edge[0] > 0 and edge[1] > 0:
        d = find_neighbour(edge, triangle_id)
        if d < 0:
            return DS
        
    elif node_id > 0 and edge[0] < 0 or edge[1] < 0:
        
        d = find_neighbour(edge, triangle_id)
        e = min(edge[0], edge[1])
        
        if d < e:
            return DS
        
        else: 
            list_flip_result = flip(edge, node_id, d, triangle_id, DS)
            new_edge1 = [edge[0], d]
            new_edge2 = [edge[1], d]
        
            legalize(node_id, new_edge1, list_flip_result[0], DS)
            legalize(node_id, new_edge2, list_flip_result[1], DS)
        
    elif node_id < 0 and edge[0] < 0 or edge[1] < 0:
        e = min(edge[0], edge[1])
        if node_id < e:
            return DS
        else:
            list_flip_result = flip(edge, node_id, d, triangle_id, DS)
            new_edge1 = [edge[0], d]
            new_edge2 = [edge[1], d]
        
            legalize(node_id, new_edge1, list_flip_result[0], DS)
            legalize(node_id, new_edge2, list_flip_result[1], DS)
    
    else:
        if_cloclwise = [clockwise([node_id, edge[0], edge[1]])[0], [node_id, edge[0], edge[1]]]
        if not if_clockwise[0]:
            if_clockwise = [clockwise([node_id, edge[1], edge[0]]), [node_id, edge[1], edge[0]]]
        
            triangle = if_clockwise[1]
            d = find_neighbour(edge, triangle_id)
    
            if in_circle(triangle, d)[0]:
                list_flip_result = flip(edge, node_id, d, triangle_id, DS)

                new_edge1 = [edge[0], d]
                new_edge2 = [edge[1], d]
        
                legalize(node_id, new_edge1, list_flip_result[0], DS)
                legalize(node_id, new_edge2, list_flip_result[1], DS)
        
    return DS

In [ ]:
def flip(edge, node1, node2, triangle_id, DS):
    
    delta1 = DS['triangles'][triangle_id]
    nb_id = find_neighbour(edge, triangle_id)
    delta2 = DS['triangles'][nb_id]
    
    delta3 = new_triangle([node1, node2, edge[0]], parents=[], children=[], neighbors=[])
    delta4 = new_triangle([node1, node2, edge[1]], parents=[], children=[], neighbors=[])
    
    delta1['children'].append(delta3['id'], delta4['id'])
    delta2['children'].append(delta3['id'], delta4['id'])
    
    delta3['parents'].append(delta1['id'], delta2['id'])
    delta4['parents'].append(delta1['id'], delta2['id'])
    
    return [delta3['id'], delta4['id'], DS]
    

## Main For Loop


In [ ]:
for node_id in range(1, len(nodes)-1):
    
    trs = locate(node_id)
    print(trs)
    
    if len(trs) == 1:
        
        Trl = DS['triangles'][trs[0]]
        
        tr1 = new_triangle([node_id, Trl['nodes'][0], Trl['nodes'][1]], parents=[], children=[], neighbors=[])
        tr2 = new_triangle([node_id, Trl['nodes'][0], Trl['nodes'][2]], parents=[], children=[], neighbors=[])
        tr3 = new_triangle([node_id, Trl['nodes'][1], Trl['nodes'][2]], parents=[], children=[], neighbors=[])
        
        tr1['parents'].append(Trl['id'])
        tr2['parents'].append(Trl['id'])
        tr3['parents'].append(Trl['id'])
        
        Trl['children'].append(tr1['id'], tr2['id'], tr3['id'])
        
        tr1['neighbors'].append(tr2['id'], tr3['id'])
        tr2['neighbors'].append(tr1['id'], tr3['id'])
        tr3['neighbors'].append(tr1['id'], tr2['id'])
        
        for neighbour_id in Trl['neighbors']:
            
            if share_edge(neighbour_id, tr1['id']):
                tr1['neighbors'].append(neighbour_id)
            
            if share_edge(neighbour_id, tr2['id']):
                tr2['neighbors'].append(neighbour_id)
            
            if share_edge(neighbour_id, tr3['id']):
                tr3['neighbors'].append(neighbour_id)
                
        legalize(node_id, [Trl['nodes'][0], Trl['nodes'][1]], trl['id'], DS)
        legalize(node_id, [Trl['nodes'][0], Trl['nodes'][2]], tr2['id'], DS)
        legalize(node_id, [Trl['nodes'][1], Trl['nodes'][2]], tr3['id'], DS)
                
    elif len(trs) == 2:
        Tr1 = DS['triangles'][trs[0]]['id']
        Tr2 = DS['triangles'][trs[1]]['id']
        
        common_nodes = find_common_nodes(Tr1['nodes'], Tr2['nodes'])
        
        n1 = set(Tr1['nodes']) - set(Tr2['nodes'])
        n2 = set(Tr2['nodes']) - set(Tr2['nodes'])
        
        tr1 = new_triangle([node_id, common_nodes[0]['id'], n1], parents=[], children=[], neighbors=[])
        tr2 = new_triangle([node_id, common_nodes[1]['id'], n1], parents=[], children=[], neighbors=[])
        
        tr1['parent'].append(Tr1['id'])
        tr2['parent'].append(Tr1['id'])
        
        Tr1['children'].append(tr1['id'], tr2['id'])
        
        tr3 = new_triangle([node_id, common_nodes[0]['id'], n2], parents=[], children=[], neighbors=[])
        tr4 = new_triangle([node_id, common_nodes[1]['id'], n2], parents=[], children=[], neighbors=[])
        
        tr3['parent'].append(Tr2['id'])
        tr4['parent'].append(Tr2['id'])
        
        Tr2['children'].append(tr3['id'], tr4['id'])
        
        tr1['neighbors'].append(tr2['id'], tr3['id'])
        for neighbour_id in Tr1['neighbors']:
            if share_edge(neighbour_id, tr1['id']):
                tr1['neighbor'].append(neighbour_id)
                   
        tr2['neighbors'].append(tr1['id'], tr4['id'])
        for neighbour_id in Tr1['neighbors']:
            if share_edge(neighbour_id, tr2['id']):
                tr2['neighbor'].append(neighbour_id)
                   
        tr3['neighbors'].append(tr4['id'], tr1['id'])
        for neighbour_id in Tr2['neighbors']:
            if share_edge(neighbour_id, tr3['id']):
                tr3['neighbor'].append(neighbour_id)
        
        
        tr4['neighbors'].append(tr3['id'], tr2['id'])
        for neighbour_id in Tr2['neighbors']:
            if share_edge(neighbour_id, tr4['id']):
                tr4['neighbor'].append(neighbour_id)
                
        l1 = tr1['nodes'].remove(node_id)
        l2 = tr2['nodes'].remove(node_id)
        l3 = tr3['nodes'].remove(node_id)
        l4 = tr4['nodes'].remove(node_id)
        
        legalize(node_id, [l1[0], l1[1]], Trl['id'], DS)
        legalize(node_id, [l2[0], l2[1]], Trl['id'], DS)
        legalize(node_id, [l3[0], l3[1]], Trl['id'], DS)
        legalize(node_id, [l4[0], l4[1]], Trl['id'], DS)

Delaunay = []

for i in range(len(DS['triangles'])):
    ii = DS['triangles'][i]['children']
    if len(ii) == 0:
        Delaunay.append(ii)

In [ ]:
DS['triangles']
                        